In [35]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")

In [36]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun
from langchain_community.vectorstores import Chroma

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.tools.retriever import create_retriever_tool
from langchain.tools import tool

from langchain import hub
from platform import python_version

from langchain.agents import create_tool_calling_agent, AgentExecutor

In [37]:
wikipedia_tool = WikipediaQueryRun(api_wrapper= WikipediaAPIWrapper())

In [38]:
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
)

In [39]:
vectorstore = Chroma(persist_directory='./test',
                     embedding_function= embeddings)

retriever = vectorstore.as_retriever(search_type = 'mmr',
                                     search_kwargs = {'k': 3, 'lambda_mult': 0.7})

retriever_tool = create_retriever_tool(retriever=retriever,
                                       name='Intro to data and data science',
                                       description='''For any questions regarding
                                       the Introduction to data and data science, you must use this tool. ''')

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [40]:
@tool("get_python_version")
def get_python_version() -> str:
    '''Useful for questions regarding the version of Python currenlty used in this project.'''
    return python_version()

In [41]:
tools = [wikipedia_tool, retriever_tool, get_python_version]

In [42]:
chat = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
)

In [43]:
chat_prompt_template = hub.pull("hwchase17/openai-tools-agent")

c:\Users\SS\anaconda3\envs\ai\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [44]:
agent = create_tool_calling_agent(llm = chat,
                                  tools = tools,
                                  prompt = chat_prompt_template)

In [45]:
agent_executor = AgentExecutor(agent = agent,
                               tools = tools,
                               verbose = True,
                               return_intermediate_steps= True)

In [ ]:
response = agent_executor.invoke({"input": "Could you tell me the version of Python I'm currenlty using?"})
# It apparently has prob with gemini for naming convention with tools. So use OpenAI API.



> Entering new AgentExecutor chain...


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.tools[0].function_declarations[1].name: Invalid function name. Must start with a letter or an underscore. Must be alphameric (a-z, A-Z, 0-9), underscores (_), dots (.) or dashes (-), with a maximum length of 64.


In [47]:
res = agent_executor.invoke({"input": '''
I'm following the Introduction to Data and Data Science course.
Could you list the programming languages a data scientist should know?
Additionally, could you tell me who their creators are?
'''})



> Entering new AgentExecutor chain...


ChatGoogleGenerativeAIError: Invalid argument provided to Gemini: 400 * GenerateContentRequest.tools[0].function_declarations[1].name: Invalid function name. Must start with a letter or an underscore. Must be alphameric (a-z, A-Z, 0-9), underscores (_), dots (.) or dashes (-), with a maximum length of 64.
